In [1]:
import time

import numpy as np

import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, BatchNormalization, AveragePooling2D, Softmax, Reshape
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.losses import Huber
from tensorflow.keras.metrics import AUC, BinaryAccuracy, Accuracy, Recall, Precision
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, LearningRateScheduler, ReduceLROnPlateau

from HWlayer_base import evaluate_build, focus_build
from HWlayer_keras import HWlayer

In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train = x_train / 255
y_train = tf.keras.utils.to_categorical(y_train, 10)

x_test = x_test / 255
y_test = tf.keras.utils.to_categorical(y_test, 10)

In [3]:
evaluate_num = 8
focus = 0.8
deepth = 3

evaluate_dims = evaluate_num * x_test.shape[-1]

In [4]:
# def evaluate_build_AveragePooling2D(data, evaluate_num=8, focus=0.8, deepth=3):
#     output_list = []
#     x = Input(data.shape[1:])
#     y_list = [x]
#     for idx in range(deepth):
#         y_list += [AveragePooling2D(2)(y_list[-1])]
    
#     model = Model(x, y_list)
#     data_list = model.predict(data)
    
#     output_list = []
#     for data in data_list:
#         print(data.shape)
#         evaluate_list = [evaluate_build(data[..., i], evaluate_num) for i in range(data.shape[-1])]
#         evaluate_focus_list = [focus_build(evaluate, focus) for evaluate in evaluate_list]
#         output_list.append(evaluate_focus_list)
        
#     return output_list

# evaluate_focus_list = evaluate_build_AveragePooling2D(x_test, evaluate_num, focus, deepth)

In [5]:
# x = Input(shape=((32, 32, 3)))
# y = x
# hw_layer_list = []
# for evaluate_focus in evaluate_focus_list:
#     l = HWlayer(evaluate_focus)(y)
#     l = K.expand_dims(l, -2)
#     hw_layer_list.append(l)
#     y = MaxPooling2D(2)(y)

In [6]:
# VGG11_cfg = [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M']
VGG16_cfg = [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M']

In [7]:
x = Input(shape=((32, 32, 3)))
y = x

for cfg in VGG16_cfg:
    if cfg == 'M':
        y = MaxPooling2D(2, 2)(y)
    else:
        y = Conv2D(cfg, 3, 1, 'same')(y)
        y = BatchNormalization()(y)
        y = Activation('relu')(y)

y = Flatten()(y)
y = Dense(10)(y)
y = Softmax(axis=-1)()
        
model = Model(x, y)
model.summary()

TypeError: ('Keyword argument not understood:', 'Activation')

In [ ]:
model.compile(loss=Huber(), optimizer=SGD(learning_rate=0.01), metrics=[ 'acc', 'AUC'])

In [ ]:
datagen = ImageDataGenerator(
    featurewise_center = False,  # 将整个数据集的均值设为0
    samplewise_center = False,  # 将每个样本的均值设为0
    featurewise_std_normalization = False,  # 将输入除以整个数据集的标准差
    samplewise_std_normalization = False,  # 将输入除以其标准差
    zca_whitening = False,  # 运用 ZCA 白化
    zca_epsilon = 1e-06,  # ZCA 白化的 epsilon值
    rotation_range = 0,  # 随机旋转图像范围 (角度, 0 to 180)
    width_shift_range = 0.1,  # 随机水平移动图像 (总宽度的百分比) 
    height_shift_range = 0.1,  # 随机垂直移动图像 (总高度的百分比)
    shear_range = 0.,  # 设置随机裁剪范围
    zoom_range = 0.,  # 设置随机放大范围
    channel_shift_range = 0.,  # 设置随机通道切换的范围
    fill_mode = 'nearest',  # 设置填充输入边界之外的点的模式
    cval = 0.,  # 在 fill_mode = "constant" 时使用的值
    horizontal_flip = True,  # 随机水平翻转图像
    vertical_flip = False,  # 随机垂直翻转图像
    rescale = None,  # 设置缩放因子 (在其他转换之前使用)
    preprocessing_function = None,  # 设置将应用于每一个输入的函数
    data_format = None,  # 图像数据格式，"channels_first" 或 "channels_last" 之一
    validation_split = 0.0)  # 保留用于验证的图像比例（严格在0和1之间）

# 计算特征标准化所需的计算量
# (如果应用 ZCA 白化，则为 std，mean和主成分).
datagen.fit(x_train)

In [ ]:
model_name = time.strftime('CIFAR10_VGG_KERAS_HWNET%Y%m%d%H%M%S')

callback_list = [
    ReduceLROnPlateau(monitor='loss', factor=0.7,  patience=4, min_lr=1e-7, verbose=True),
    EarlyStopping(monitor='loss', patience=32),
    ModelCheckpoint('models/%s_best.h5'%model_name, monitor='loss', save_best_only=True, verbose=False),
    TensorBoard(log_dir='./Log/%s'%model_name)
]

model.fit(datagen.flow(x_train, y_train, batch_size=100), epochs=200, validation_data=(x_test, y_test), callbacks=callback_list)